/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_checkin.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/Dataset_User_Agreement.pdf
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_tip.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_review.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_business.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_user.json


In [2]:
# import the data (chunksize returns jsonReader for iteration)
businesses = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=2000000)
reviews = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_review.json", lines=True, orient='columns', chunksize=2000000)
users = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_user.json", lines=True, orient='columns', chunksize=2000000)

In [3]:
# read data 
for business in businesses:
    subset_business = business[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'stars', 'review_count', 'is_open', 'attributes', 'categories']]
    break
    
for review in reviews:
    subset_review = review[['review_id', 'user_id', 'business_id', 'stars', 'date']]
    break

for user in users:
    subset_user = user[['user_id', 'name', 'review_count', 'yelping_since', 'average_stars']]
    break

In [5]:
print(subset_business.shape)
print(subset_review.shape)
print(subset_user.shape)
# peak the tables
display(subset_business.head(2))
display(subset_review.head(2))
display(subset_user.head(2))

(150346, 11)
(2000000, 5)
(1987897, 5)


,business_id,name,address,city,state,postal_code,stars,review_count,is_open,attributes,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."


,review_id,user_id,business_id,stars,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18


,user_id,name,review_count,yelping_since,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,3.74


In [18]:
pd.DataFrame(np.unique(subset_business['city']))

,0
0,AB Edmonton
1,AMBLER
2,ARDMORE
3,AVON
4,Abington
...,...
1411,wilmington
1412,wimauma
1413,​Clayton
1414,​Largo


In [16]:
# subset_business[subset_business['business_id'=='XQfwVwDr-v0ZS3_CbbE5Xw']]
print(subset_business[subset_business['business_id']=='XQfwVwDr-v0ZS3_CbbE5Xw']['attributes'])

1867    {'NoiseLevel': 'u'average'', 'HasTV': 'False',...
Name: attributes, dtype: object
